In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from utils.tuning import combine_csvs
import config

In [ ]:
# Load split metadata
df_split_metadata = pd.read_csv(Path(config.output_dir_splits) / "split_metadata.csv")
df_split_metadata["split"] = df_split_metadata["filepath"].apply(lambda x: Path(x).stem)

In [ ]:
model_name = "mae"
df = combine_csvs("output/tuning/" + model_name , out_name=f"/{model_name}_tuning.csv", remove_files=False)
df["total_time"] = df["train_time"] + df["pred_time"]
df = df.merge(df_split_metadata, on="split", how="left")

In [ ]:
print(len(df))

In [ ]:
df.columns

In [ ]:
split_resolution_map = {"fold_block_0": "Block (5°)", "fold_block_15": "Block (15°)", "fold_block_30": "Block (40°)", "fold_random_0": "Random (10%)", "fold_random_1": "Random (20%)"}
df["split_plot_name"] = df["split"].map(split_resolution_map)
df["scheme"] = df["scheme"].map({"random": "Random", "block": "Block"})
df = df.rename(columns={"scheme": "Scheme"})

In [ ]:
print(f"{model_name}: Mean total time per model = {df.total_time.mean():.2f} s, mean validation RMSE per model = {df.val_rmse.mean():.5f} +- {df.val_rmse.std():.5f}")
print(f"{model_name}: Mean train time per model = {df.train_time.mean() / 60:.2f} min")

sns.boxplot(x=0, y=df["val_rmse"])
plt.ylabel("Validation RMSE")
plt.xticks(ticks=[0], labels=[f"{model_name} imputation"])
plt.show()

sns.boxplot(y=df["train_time"], x=0)
if df["pred_time"].any(): sns.boxplot(y=df["pred_time"], x=1)
plt.xticks(ticks=[0, 1], labels=["Train", "Predict"])
plt.ylabel("Time [s]")
plt.show()

plt.figure(figsize=(7, 5))
sns.boxplot(df, x="split_plot_name", y="val_rmse", hue="Scheme")#, size="val_size")
# plt.xticks(ticks=df["split_plot_name"], labels=df["filepath"].astype("category").cat.codes)
# plt.xticks(ticks=df["split_plot_name"], labels=df["split_plot_name"])
plt.ylabel("Validation RMSE")
plt.xlabel("Splitting scheme")
plt.savefig("output/high_res_plots/validation_rmse_over splits_" + model_name + ".png", dpi=1000)
plt.show()

In [ ]:
df[["val_rmse", "train_time", "pred_time", "val_size"]].corr()

In [ ]:
if model_name == "mean":
    df_mean = df[df["hyps"].str.contains("mean")]
    df_median = df[df["hyps"].str.contains("median")]
    print("Difference in RMSE between mean and median imputation: ", df_mean.val_rmse.mean()-df_median.val_rmse.mean())

In [ ]:
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from utils.tuning import load_optuna_study

# model_name = "mae"
# Load Optuna study
study = load_optuna_study(model_name)

In [ ]:
best_params = study.best_trial.params
print("Hyperparameters: ", best_params)
print("Validation RMSE: ", study.best_trial.value)
print("Duration: ", study.best_trial.duration)
print(study.best_trial)

In [ ]:
trial_times = [trial.value for trial in study.trials]
plt.figure(figsize=(6, 4))
plt.boxplot(trial_times, vert=True, patch_artist=True)
plt.ylabel("Validation RMSE per trial")
plt.title("Optuna Trial RMSE")
plt.show()

In [ ]:
trial_times = [trial.duration.total_seconds() for trial in study.trials]
plt.figure(figsize=(6, 4))
plt.boxplot(trial_times, vert=True, patch_artist=True)
plt.ylabel("Trial Time (seconds)")
plt.title("Optuna Trial Times")
plt.show()

In [ ]:
# Feature importances
optuna.importance.get_param_importances(study)

In [ ]:
# Set running trials to FAIL
# study._storage.remove_running_trials(study._study_id)

In [ ]:
if model_name == "mae":
    invalid_durations = pd.DataFrame({"trial": [37, 42, 47, 56, 62, 67], "fold": ["fold_random_0", "fold_block_0", "fold_block_0", "fold_block_15", "fold_block_30", "fold_block0"]})
else:
    invalid_durations = pd.DataFrame(columns=["trial", "fold"])

durations = [
    {
        "trial": x.number,
        "duration": x.duration.seconds / 60 if x is not None and x.duration is not None and x.number not in list(invalid_durations.trial) else None
    }
    for x in study.trials
]
durations = pd.DataFrame(durations)

sns.lineplot(durations, x="trial", y="duration")

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_slice(study)

In [ ]:
target_dpi = 1000

fig_contour = plot_contour(study)
if model_name == "mice": fig_contour.update_layout(width=600, height=600)
elif model_name == "knn": fig_contour.update_layout(width=800, height=600)
elif model_name == "mae" or model_name == "mae_finetune": fig_contour.update_layout(width=1200, height=1600)
else: fig_contour.update_layout(width=600, height=800)
fig_contour.update_layout(title=None)
fig_contour.write_image(f"output/high_res_plots/contour_{model_name}.png", scale=target_dpi/96)
fig_contour.show()


In [ ]:
# Mean and std of RMSE across trials
values = [t.value for t in study.get_trials() if t.state == optuna.trial.TrialState.COMPLETE]
df_rmse = pd.DataFrame(values, columns=["rmse"])
mean_error = df_rmse["rmse"].mean()
std_error = df_rmse["rmse"].std()

print("Mean:", mean_error)
print("Std:", std_error)

# Train model on best hyperparameter combination

In [4]:
import json
import numpy as np
from sklearn.base import BaseEstimator
import torch
import pandas as pd

from tuning_studies.sklearn_study import train_sklearn_single_split
from tuning_studies.pytorch_study import train_mae_single_split
from utils.tuning import set_seed, TuningResult, get_model_class, load_optuna_study
from nn_utils.dataset import load_dataset
import config

In [5]:
# Load test-train split
split = json.load(open(f"{config.output_dir_splits}/test_train_split.json"))
test_idx = np.array(split["test_idx"], dtype=int)
train_idx = np.array(split["train_idx"], dtype=int)

# Load dataset
df = load_dataset()

# Load study
model_name = "mae"
model_class = get_model_class(model_name)
study = load_optuna_study(model_name)
hyps = study.best_trial.params

# Set seed
seed = 42
set_seed(seed=seed)

Study loaded as db


In [6]:
hyps["d_model"]

128

In [ ]:
# Assemble hyps for mae
if model_name == "mae":
    hyps = {"train": {
                "batch_size": hyps["batch_size"],
                "learning_rate": hyps["lr"],
                "patience": 5,
                "n_epochs": 20,
                "mask_ratio": hyps["mask_ratio"],
                "loss": hyps["loss"],
                "optimizer": torch.optim.Adam
            },
            "model": {
                "d_model": hyps["d_model"],
                "nhead": hyps["nhead"],
                "nlayers": hyps["nlayers"],
                "dim_feedforward": hyps["dim_feedforward"],
                "dropout": hyps["dropout"],
            }
        }

In [ ]:
i = 0

# for i in range(10):  # Repeat to assess robustness of predictions? @todo
if issubclass(model_class, BaseEstimator):
    print("Sklearn model...")
    results, y_true, y_pred = train_sklearn_single_split(
        df=df, model_class=model_class, hyps=hyps,
        test_idx=test_idx, train_idx=train_idx, val_idx=test_idx,
        model_name=model_name, split_path="final",
        trial_id=i, optuna_callback=None, seed=seed+i,
        save_model=True)

elif issubclass(model_class, torch.nn.Module):
    print("PyTorch model...")
    results, y_true, y_pred, variance = train_mae_single_split(
        df=df, model_class=model_class, hyps=hyps,
        train_idx=train_idx, val_idx=test_idx, test_idx=test_idx,
        model_name=model_name, split_path="final",
        trial_id=i, optuna_callback=None, seed=seed+i,
        tuning_mode=False, device=torch.device("cuda"),
        save_model=True)

else:
    print(f"Unknown model type: {model_class}")


In [ ]:
np.nanmean(variance)

In [ ]:
import seaborn as sns
sns.boxplot(variance)

In [ ]:
# Store y_pred
df_pred = pd.DataFrame(y_pred, columns=config.parameters)
df_pred.to_csv(f"output/tuning/{model_name}/model{model_name}_y_pred.csv")

In [ ]:
pd.DataFrame(y_true, columns=config.parameters).to_csv(f"output/tuning/{model_name}/model{model_name}_y_true.csv")

In [ ]:
(np.nanmean((y_true - y_pred) ** 2))

In [ ]:
ma = np.abs(np.nanmean((y_true - y_pred)))

In [ ]:
err = np.abs(y_true - y_pred)

In [ ]:
err.shape

In [ ]:
variance.shape

In [ ]:
col = 3
np.corrcoef(err[:, col], variance[:, col])

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(err.flatten(), variance.flatten())
plt.xlabel("Error")
plt.ylabel("Variance")
plt.show()

In [ ]:
from models.mae import OceanMAE
import joblib
import json

# Load model
with open(f"output/tuning/{model_name}/model{model_name}_splitfinal_trial0.json") as f:
    meta = json.load(f)

if meta["model_framework"] == "sklearn":
    model = joblib.load(meta["model_path"])
elif meta["model_framework"] == "pytorch":
    model = OceanMAE(**meta["hyps"]["model"])
    model.load_state_dict(torch.load(meta["model_path"]))
    model.eval()

In [ ]:
meta